In [0]:
spark.sql("USE CATALOG hive_metastore")
storage_account = "databricks180820251"

In [0]:
%sql
create or replace table retail_silver.orders_source
select 
order_id,
dateadd(order_date,4115) as order_date,
customer_id,
order_status
from retail_silver.orders


In [0]:
%sql
select 
min(order_date) as mindate,
max(order_date) as maxdate
from retail_silver.orders_source

# One Time load

In [0]:
df = spark.sql("select * from retail_silver.orders_source where order_date < curdate()")

In [0]:
%sql
drop table retail_silver.orders_tgt

In [0]:
df.write.option("path", f"abfss://silver@{storage_account}.dfs.core.windows.net/retail/orders_tgt").mode("overwrite").saveAsTable("retail_silver.orders_tgt")

In [0]:
%sql
select 
min(order_date) as mindate,
max(order_date) as maxdate
from retail_silver.orders_tgt

In [0]:
%sql
select 
count(*)
from retail_silver.orders_tgt

In [0]:
%sql
select count(*) from retail_silver.orders_source where order_date = current_date()
185+68120

In [0]:
from delta.tables import *
from pyspark.sql.functions import lit,col
orders=DeltaTable.forPath(spark, f"abfss://silver@{storage_account}.dfs.core.windows.net/retail/orders_tgt")

In [0]:



df = spark.read.format("delta").load(f"abfss://silver@{storage_account}.dfs.core.windows.net/retail/orders_tgt")
df.count()

# Incremental load Append

In [0]:
df = spark.sql("select * from retail_silver.orders_source where order_date = curdate()")

In [0]:
df.count()

In [0]:
df.write.option("path", f"abfss://silver@{storage_account}.dfs.core.windows.net/retail/orders_tgt").mode("append").saveAsTable("retail_silver.orders_tgt")

In [0]:
%sql
select count(*) from retail_silver.orders_tgt

In [0]:
%sql
select max(order_date)  from retail_silver.orders_tgt